In [4]:
# Utilities for converting ISO 8601 datetime format

from datetime import datetime

def _to_unix_time(iso_time) -> int:
    # The Python datetime library doesn't support the military timezone suffixes
    # (which actually violates the ISO 8601 standard).
    strip_time_zone = iso_time.replace('Z', '+00:00')
    dt = datetime.fromisoformat(strip_time_zone)
    return(int(dt.timestamp()))
    

In [5]:
# Interface for dumping data to the Netscape bookmark format

# Couple things to note in this format:
# The <DT> tag doesn't seem to have a closing tag
# The <DD> tag can be used to hold description information about a particular item. (It also
# doesn't seem to have a closing tag.)

import os
from pathlib import Path
from datetime import date

def _create_entry_from_playlist(p_item):
    '''
    @param PlaylistItem
    '''
    title = p_item.title
    updated_at = _to_unix_time(p_item.updated_at)
    url = p_item.url

    return f'<DT><A HREF="{url}" ADD_DATE="{updated_at}">{title}</A>'


def _create_entry_from_channel(c_item):
    '''
    @param ChannelItem
    '''
    title = c_item.channel
    updated_at = _to_unix_time(c_item.published_at)
    url = c_item.url
    
    return f'<DT><A HREF="{url}" ADD_DATE="{updated_at}">{title}</A>'
    

class Bookmarks(object):
    
    def __init__(self, file_name='Bookmark.html', file_path=None):
        _today = date.today()
        self.file_name = file_name
        self.file_path = f'data/{_today}' if file_path is None else file_path
        self.full_path = os.path.join(self.file_path, self.file_name)
                
        # Delete the bookmark file if it exists
        # A fresh start will prevent clashes when new items are appended
        if os.path.exists(self.full_path):
            os.remove(self.full_path)
        
        # Make the directories if they don't exist
        Path(self.file_path).mkdir(parents=True, exist_ok=True)
        
        with open(self.full_path, 'a+') as fp:
            print('<!DOCTYPE NETSCAPE-Bookmark-file-1>', file=fp)
            print('<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=UTF-8">', file=fp)
            print('<TITLE>Bookmarks</TITLE>', file=fp)
            print('<H1>Bookmarks Menu</H1>', file=fp)
            print('<DL><p>', file=fp)
    
    def _validate(collection_item):
        pass
        

    # Collection = 2 tuple comprising:
    # collection_name = YouTube playlist name
    # collection_entries = list of PlaylistItem objects
    def add_playlist_items(self, collection):
        collection_name, collection_entries = collection
        
        with open(self.full_path, 'a+') as fp:
            print(f'<DT><H3>{collection_name}</H3>', file=fp)
            print('<DL><p>', file=fp)
            for i in collection_entries:
                print(_create_entry_from_playlist(i), file=fp)

            print('</DL><p>', file=fp)

            
    # collection = list of ItemChannelItemobjects
    def add_subscription_items(self, collection):
        
        collection_name = 'YouTube Subscriptions'
        
        with open(self.full_path, 'a+') as fp:
            print(f'<DT><H3>{collection_name}</H3>', file=fp)
            print('<DL><p>', file=fp)
            for i in collection:
                print(_create_entry_from_channel(i), file=fp)

            print('</DL><p>', file=fp)
        
    
    def flush(self):
        with open(self.full_path, 'a+') as fp:
            print('</DL>', file=fp)
        

In [12]:
bk = Bookmarks()